In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
import numpy as np
import pandas as pd

from datetime import date, timedelta
import luigi
import os
import logging

from bt_ai.stable.data_input.user_trait_store import UserTraitsUnloadTask
from bt_ai.stable.data_input.dataframe import DataFrameTarget

from notebook_utils.logging import setup_logging  #pip3 install bt-notebook-utils
from notebook_utils.luigi import run_luigi_tasks

In [6]:
import warnings
warnings.filterwarnings(action='ignore')

pd.options.display.max_columns = 999

In [7]:
luigi_planner_uri = 'http://localhost:8082'

logging_overrides = {
    'luigi-interface': logging.INFO,
}

LOG = logging.getLogger('jupyter')

In [8]:
class TrainingUsers(luigi.ExternalTask):
    path = luigi.Parameter()

    def output(self):
        root_path = os.path.dirname(self.path)
        filename = os.path.basename(self.path)

        return DataFrameTarget(
            root_path=root_path,
            column_names=[],
            filenames=[filename],
            field_sep=',',
            compute=True,
            compression=None
        )

In [9]:
users = TrainingUsers(
    path='s3://manish-sandbox/ml-dumps/zcode_project/large-scale/email_md5_df_sample200.csv'
)

In [16]:
end_date = date(2019, 9, 30)
start_date = end_date - timedelta(days=30)
s3_bucket = 'kalyan-sandbox' # change this to point to your sandbox.
spark_job_name = 'kalyan_Sprint_PlaceIQ_Pyarrow'#change this to your name, so that you identify your launched cluster.

- 'm5.4xlarge' is not allowed in 1e aws_az

In [25]:
partition_date = date(2019, 7, 28)

user_traits = UserTraitsUnloadTask(
    site_id='sprint',
    partition_date=partition_date,
    spark_job_name=spark_job_name,
    instance_type='r5.4xlarge',
    core_count=48,
    use_spots=True,
    aws_az = 'us-east-1a',
    keep_alive=True,
    write_partitions=200,
    
    staging_bucket=s3_bucket,
    unload_bucket=s3_bucket,
    dependencies=[users]
)

In [27]:
# do not forget to initiat luigi from terminal by typing 'luigid'
run_luigi_tasks([user_traits], scheduler_uri=luigi_planner_uri)

Cluster will not auto shutdown.  Please ensure you shut it down manually
Cluster will not terminate when job finishes.  Please ensure you shut it down manually.
Cluster has termination protection set; auto shutdown will be disabled.  Please ensure you shut it down manually.
